In [4]:
import numpy as np
import torch
import matplotlib.pyplot as plt


In [3]:
np.load('/home/e19b516g/yejing/data/data_for_graph/S150_R10/val/los/N4E8_UN_102_em_35.npy')

array([[0., 1., 1., 1.],
       [1., 0., 1., 0.],
       [1., 1., 0., 0.],
       [1., 0., 0., 0.]])

In [3]:
s1 = "435 181, 437 182, 437 184, 437 188, 437 191, 437 194, 437 198, 436 202, 436 206, 436 210, 435 215, 435 220, 435 224, 435 229, 434 232, 434 236, 434 239, 434 242, 435 244, 435 246, 435 247, 435 248, 435 249, 436 249, 436 248, 437 246, 437 244, 438 242, 439 240, 441 237, 443 234, 445 231, 448 228, 452 225, 455 223, 457 221, 461 220, 464 219, 467 218, 471 218, 474 218, 477 220, 479 222, 482 224, 484 227, 486 229, 488 232, 489 235, 489 238, 489 241, 488 244, 487 248, 486 251, 484 254, 482 257, 479 259, 475 262, 471 264, 467 266, 463 267, 457 268, 452 268, 447 268, 444 268, 440 267, 437 266, 434 265, 432 264, 431 262, 430 261, 430 260"
s2 = "500 165, 498 177, 498 180, 498 182, 498 184, 498 186, 497 187, 497 188, 497 189, 497 190, 496 189, 496 188, 496 187, 496 185, 496 183, 497 181, 498 179, 499 176, 500 174, 501 171, 502 169, 503 167, 504 166, 506 164, 508 163, 509 163, 511 163, 513 163, 514 164, 515 166, 516 168, 517 170, 517 172, 518 175, 517 177, 517 180, 517 182, 516 184, 516 186, 515 188, 515 190, 514 191, 514 192, 514 193, 513 194, 513 195"
def make_coor(coordinates_str):
    coordinates_list = [map(int, coord.split()) for coord in coordinates_str.split(', ')]
    coordinates_tuples = [(x, y) for x, y in coordinates_list]
    return coordinates_tuples
s1 = make_coor(s1)
s2 = make_coor(s2)
print(s1)
s1 = np.array(s1)
s2 = np.array(s2)
# minx = min(np.min(s1, axis=0)[0], np.min(s2, axis=0)[0])
# miny = min(np.min(s1, axis=0)[1], np.min(s2, axis=0)[1])
ss = np.concatenate((s1, s2), axis=0)
d = np.max(ss, axis=0) -np.min(ss, axis=0)
s1_norm = (s1 - np.min(ss, axis=0)) / d
s2_norm = (s2 - np.min(ss, axis=0)) / d


[(435, 181), (437, 182), (437, 184), (437, 188), (437, 191), (437, 194), (437, 198), (436, 202), (436, 206), (436, 210), (435, 215), (435, 220), (435, 224), (435, 229), (434, 232), (434, 236), (434, 239), (434, 242), (435, 244), (435, 246), (435, 247), (435, 248), (435, 249), (436, 249), (436, 248), (437, 246), (437, 244), (438, 242), (439, 240), (441, 237), (443, 234), (445, 231), (448, 228), (452, 225), (455, 223), (457, 221), (461, 220), (464, 219), (467, 218), (471, 218), (474, 218), (477, 220), (479, 222), (482, 224), (484, 227), (486, 229), (488, 232), (489, 235), (489, 238), (489, 241), (488, 244), (487, 248), (486, 251), (484, 254), (482, 257), (479, 259), (475, 262), (471, 264), (467, 266), (463, 267), (457, 268), (452, 268), (447, 268), (444, 268), (440, 267), (437, 266), (434, 265), (432, 264), (431, 262), (430, 261), (430, 260)]


In [21]:
import math
PI = math.pi

def fuzzy_se(op, ua):
    return max(0, 1 -  (2/PI)*np.arccos(op.dot(ua)/np.linalg.norm(op)))

def fuzzy_stroke(stroke, point_o, ua):
    # print(stroke)
    fuzzy_stroke = []
    for point in stroke:
        x_p = point[0]
        y_p = point[1]
        op = np.array([x_p - point_o[0], y_p - point_o[1]])
        v_p = fuzzy_se(op, ua)
        fuzzy_stroke.append(v_p)
    # print(fuzzy_stroke)
    return np.array(fuzzy_stroke).astype(np.float32)

def calculate_center(stroke):
    left = math.inf
    right = -math.inf 
    top = -math.inf 
    bottom = math.inf 
    for x, y in stroke:
        left = min(left, x)
        right = max(right, x)
        top = max(top, y)
        bottom = min(bottom, y)
    center_x = (left + right) / 2
    center_y = (top + bottom) / 2
    return [center_x, center_y]

In [112]:
from PIL import Image, ImageDraw
width, height = 300, 300
# grey_values = np.random.rand(height, width)
# scaled_grey_values = grey_values * 255
ua_0 = np.array([0, 1])
ua_1 = np.array([0, -1])
ua_2 = np.array([1, 0])
ua_3 = np.array([-1, 0])
scaled_grey_values = np.zeros((width, height))
s1_draw = [(int(x * width), int(y * height)) for x, y in s1_norm]
s2_draw = [(int(x * width), int(y * height)) for x, y in s2_norm]


In [138]:
def make_image(s1_draw, s2_draw, ua,name):
    center_s1 = calculate_center(s1_draw)
    for i in range(width):
        for j in range(height):
            op = np.array([i- center_s1[1], j - center_s1[0]])
            scaled_grey_values[i,j] = int(fuzzy_se(op, ua) * 255)

    s2_sample = [s2_draw[0],s2_draw[5], s2_draw[10], s2_draw[15],s2_draw[20],s2_draw[25],s2_draw[30],s2_draw[35], s2_draw[40], s2_draw[45],s2_draw[-1]]

    grey_image = Image.fromarray(scaled_grey_values.astype('uint8'), mode='L')
    rgb_image = grey_image.convert('RGB')
    # image = Image.new('L', (width, height), scaled_grey_values)
    draw = ImageDraw.Draw(rgb_image)
    draw.line(s1_draw, fill='red', width=3)
    draw.line(s2_draw,fill='blue', width=3)
    center = (center_s1[0] - 2, center_s1[1] - 2, center_s1[0] + 2, center_s1[1] + 2)
    s2_sample = [s2_draw[0],s2_draw[7], s2_draw[14], s2_draw[21],s2_draw[28],s2_draw[35],s2_draw[42],s2_draw[49], s2_draw[56], s2_draw[63],s2_draw[-1]]

    # draw.ellipse(center, fill='yellow', width=2)
    for point in s2_sample:
        bounding_box = (point[0] - 2, point[1] - 2, point[0] + 2, point[1] + 2)
        draw.ellipse(bounding_box, fill='yellow', width=2)
    rgb_image.save(str(name) + ".png")
    # rgb_image.show()


In [139]:
make_image(s2_draw, s1_draw, ua_0, 'b_ua_0')
make_image(s2_draw, s1_draw, ua_1, 'b_ua_1')
make_image(s2_draw, s1_draw, ua_2, 'b_ua_2')
make_image(s2_draw, s1_draw, ua_3, 'b_ua_3')




In [2]:
from post_traintement import *
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
results_path = '/home/e19b516g/yejing/code/Edge_GAT/results/test'
all_am = []
all_gt = []
all_node_pred = torch.zeros(0)
all_node_gt = torch.zeros(0)
for root, _, files in os.walk(results_path):
        for file in files:
            if file.endswith('.pt'):
                path = os.path.join(root, file)
                stroke_emb, edge_emb, stroke_label, edge_label = load_pt(path)
                seg_am = torch.where(torch.argmax(edge_emb, dim=2) == 1, 1, 0)
                seg_gt = torch.where(edge_label == 1, 1, 0)
                all_node_pred = torch.cat((all_node_pred, torch.argmax(stroke_emb, dim=1)))
                all_node_gt = torch.cat((all_node_gt, stroke_label))
                for i in range(seg_am.shape[0]-1):
                    all_am.append(int(seg_am[i,i+1]))
                    all_gt.append(int(seg_gt[i,i+1]))

In [5]:
print(accuracy_score(np.array(all_node_pred), np.array(all_node_gt)))
# print(precision_score(np.array(all_node_pred), np.array(all_node_gt)))
print(recall_score(np.array(all_node_pred), np.array(all_node_gt), average='weighted'))
# print(f1_score(np.array(all_node_pred), np.array(all_node_gt)))

0.8500402950839997


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [169]:
print(accuracy_score(np.array(all_am), np.array(all_gt)))
print(precision_score(np.array(all_am), np.array(all_gt)))
print(recall_score(np.array(all_am), np.array(all_gt)))
print(f1_score(np.array(all_am), np.array(all_gt)))

0.9632038488952245
0.9143480632842335
0.954985754985755
0.9342251950947603
